In [13]:
import pandas as pd
import plotly.express as px

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import Adam
from keras.losses import mae


### Import the dataset and processing data

In [14]:
df = pd.read_csv("https://raw.githubusercontent.com/B11070043/Predict-price/main/Dataset/Gold_train.csv?token=GHSAT0AAAAAACH7JK6JYGWWYTBTWTCS4PGGZIPCKAQ", index_col='date')
df.head()


,open,high,low,close,volume
date,,,,,
2000-08-30,273.899994,273.899994,273.899994,273.899994,0
2000-08-31,274.799988,278.299988,274.799988,278.299988,0
2000-09-01,277.000000,277.000000,277.000000,277.000000,0
2000-09-05,275.799988,275.799988,275.799988,275.799988,2
2000-09-06,274.200012,274.200012,274.200012,274.200012,0


In [15]:
X = df[['open','high','low','close']]
Y = df[['open','close']]

In [16]:
x_train = X[:-1000]
y_train = Y[:-1000]
X_test = X[-1000:]
Y_test = Y[-1000:]
Y_test

,open,close
date,,
2019-09-16,1506.599976,1503.099976
2019-09-17,1497.900024,1505.099976
2019-09-18,1508.900024,1507.500000
2019-09-19,1496.099976,1498.400024
2019-09-20,1503.000000,1507.300049
...,...,...
2023-08-29,1917.900024,1936.500000
2023-08-30,1936.000000,1944.300049
2023-08-31,1944.400024,1938.199951


### Create model 

In [17]:
model = Sequential([
    Dense(100,input_shape=(4,),activation='relu'),
    Dense(2,activation='linear')
])

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               500       
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 702 (2.74 KB)
Trainable params: 702 (2.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Train model

In [19]:
model.compile(optimizer = Adam(learning_rate=0.0001), loss = mae, metrics = ['mae'])
history = model.fit(x_train, y_train, validation_split=0.4,steps_per_epoch=400, epochs=10,verbose=0)

In [20]:
fig = px.line(title='Mae')
fig.add_scatter(y=history.history['mae'], name='Mae')
fig.add_scatter(y=history.history['val_mae'], name='val_mae')
fig.show()


In [21]:
predictions = model.predict(X_test)

32/32 [==============================] - 0s 315us/step


In [22]:
predictions

array([[1504.7627, 1502.3955],
       [1501.843 , 1500.9211],
       [1504.6344, 1503.791 ],
       ...,
       [1943.5142, 1939.3857],
       [1946.2657, 1942.4912],
       [1967.0991, 1964.2865]], dtype=float32)

In [23]:
model.save('./Model/model.h5')

/Users/bank/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.

